# Lab 6.2: Explaining a simple OR function

This lab examines what it looks like to explain an OR function (OR = logical OR) using SHAP values. It is based on a simple example with two features `is_young` and `is_female`, roughly motivated by the Titanic survival dataset where women and children were given priority during the evacuation and so were more likely to survive. In this simulated example this effect is taken to the extreme, where all children and women survive and no adult men survive.

The goal is to compare the theoretical formulas of the SHAP values to the ones computed by the shap library.

In [ ]:
# Print the Python version
import sys
print(sys.version)

In [ ]:
# To install shap on your laptop, you can run in Spyder or in a terminal (with Jupyterlab for example)
# conda install -c conda-forge shap
# On Google colab
!pip install shap

In [ ]:
import numpy as np
import shap

### Create a dataset following an OR function

In [ ]:
N = 100 # We generate N samples for the dataset
M = 2

# randomly create binary features for (is_young, and is_female)
X = (np.random.randn(N,2) > 0) * 1

# force the first sample to be a young male
X[0,0] = 1
X[0,1] = 0

# you survive only if you are young or female
y = ((X[:,0] + X[:,1]) > 0) * 1

In [ ]:
print(X.shape)
print(y.shape)

### Train a linear regression model to mimic this OR function

In [ ]:
# a simple linear model
import sklearn
model = sklearn.linear_model.LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
print(y_pred)

### Question 1: Print the coefficients of the linear model: $f(x)=\beta_0 +\beta_1 X_1 + \beta_2 X_2$

In [ ]:
# Fill in this cell
print(model.coef_)
print(model.intercept_)

## Explain the prediction for a young boy

### Using the training set for the background distribution

Note that in the example explanation below `is_young = True` has a positive value (meaning it increases the model output, and hence the prediction of survival), while `is_female = False` has a negative value (meaning it decreases the model output). While one could argue that `is_female = False` should have no impact because we already know that the person is young, SHAP values account for the impact a feature has even when we don't necessarily know the other features, which is why `is_female = False` still has a negative impact on the prediction.

In [ ]:
explainer = shap.Explainer(model.predict, X)
shap_values = explainer(X[:1,:])
print("Model prediction:", model.predict(X[:1,:]).squeeze().round(4)) # This is the first sample
print("SHAP values for (is_young = True, is_female = False):", shap_values[0].values)
print("SHAP base values:", shap_values[0].base_values.round(4))
print("Model output with SHAP:", (shap_values[0].base_values + shap_values[0].values.sum()).round(4))
shap.plots.waterfall(shap_values[0])

### Question 2: Compute the SHAP base value on the full set


It is $E_X[f(X)]$

In [ ]:
# Fill in this cell
shap_base_value = np.mean(y_pred)
print(shap_base_value.round(4))

### Question 3: Compute the expectation for each features. Check that you can deduce from them the SHAP base value.

In [ ]:
expect_features = np.mean(X,0)

In [ ]:
# Fill in this cell
expect_weighted_features = (model.coef_)*(np.mean(X,0))
print(expect_weighted_features)

print(np.sum(expect_weighted_features)+model.intercept_)

### Question 4: Compute "by hand" the prediction of the first sample X[0,:]

In [ ]:
# Fill in this cell
prediction_by_hand = (model.coef_).dot(X[0,:])+model.intercept_
print(prediction_by_hand.round(4))

### Question 5: Compute by hand the shap values of the first sample X[0,:]. You must find exactly the values computed with the shap library.

In [ ]:
# Fill in this cell
phi = (model.coef_)*X[0,:]-expect_weighted_features
print(phi.round(4))

### Using only negative examples for the background distribution

The point of this second explanation example is to demonstrate how using a different background distribution can change the allocation of credit among the input features. This happens because we are now comparing the importance of a feature as compared to being someone who died (an adult man). The only thing different about the young boy from someone who died is that the boy is young, so all the credit goes to the `is_young = True` feature.

This highlights that often explanations are clearer when a well defined background group is used. In this case it changes the explanation from how this sample is different than typical, to how this sample is different from those who died (in other words, why did you live?).

In [ ]:
explainer = shap.Explainer(model.predict, X[y == 0,:])
# Do not forget that X[y == 0,:] only contains values like [0,0]
shap_values = explainer(X[:1,:])
print("Model prediction:", model.predict(X[:1,:]).squeeze().round(4))
print("SHAP values for (is_young = True, is_female = False):", shap_values[0].values)
print("SHAP base values:", shap_values[0].base_values.round(4))
print("Model output with SHAP:", (shap_values[0].base_values + shap_values[0].values.sum()).round(4))
shap.plots.waterfall(shap_values[0])

### Question 6: Check the shap values provided by the shap library. You must follow the same approach as in the previous questions. 

In [ ]:
# Fill in this cell
shap_base_value_negative = np.mean(model.predict(X[y == 0,:]))
print("SHAP base values:", shap_base_value_negative.round(4))
expect_features_negative = (model.coef_)*(np.mean(X[y == 0,:],0))
print("Expectation of the features:", expect_features_negative)
phi_negative = (model.coef_)*X[0,:]-expect_features_negative
print("SHAP values for (is_young = True, is_female = False):", phi_negative)

### Using only positive examples for the background distribution

We could also use only positive examples for our background distribution, and since the difference between the expected output of the model (under our background distribution) and the current output for the young boy is zero, the sum of the SHAP values will be also be zero.

In [ ]:
explainer = shap.Explainer(model.predict, X[y == 1,:])
shap_values = explainer(X[:1,:])
print("Model prediction:", model.predict(X[:1,:]).squeeze().round(4))
print("SHAP values for (is_young = True, is_female = False):", shap_values[0].values)
print("SHAP base values:", shap_values[0].base_values.round(4))
print("Model output with SHAP:", (shap_values[0].base_values + shap_values[0].values.sum()).round(4))
shap.plots.waterfall(shap_values[0])

### Question 7: Check the shap values provided by the shap library. You must follow the same approach as in the previous questions. 

In [ ]:
# Fill in this cell
shap_base_value_positive = np.mean(model.predict(X[y == 1,:]))
print("SHAP base values:", shap_base_value_positive.round(4))
expect_features_positive = (model.coef_)*(np.mean(X[y == 1,:],0))
phi_positive = (model.coef_)*X[0,:]-expect_features_positive
print("SHAP values for (is_young = True, is_female = False):", phi_positive)